In [3]:
import pandas as pd
import torch
import os
import numpy as np
import datasets
import transformers
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

from datasets import load_dataset, Dataset, DatasetDict

In [4]:
# !watch -n 0.5 nvidia-smi

In [5]:
print(f'PyTorch version: {torch.__version__}')  # 1.9.1+cu111
print(f'CUDA version: {torch.version.cuda}')  # 11.1
print(f'cuDNN version: {torch.backends.cudnn.version()}')  # 8005
print(f'Current device: {torch.cuda.current_device()}')  # 0
print(f'Is cuda available: {torch.cuda.is_available()}')  # TRUE

PyTorch version: 2.0.1
CUDA version: 11.8
cuDNN version: 8700
Current device: 0
Is cuda available: True


In [6]:
print(f'Transformers version: {transformers.__version__}')
print(f'Datasets version: {datasets.__version__}')

Transformers version: 4.37.2
Datasets version: 2.14.5


In [7]:
# Prevent a warning related to the tokenization process in the transformers library. 
os.environ["TOKENIZERS_PARALLELISM"] = "False"
# Makes CUDA operations synchronous
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [8]:
# Find the GPU with the least memory usage.
!nvidia-smi

Tue Feb 13 20:08:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        On  | 00000000:41:00.0 Off |                  N/A |
| 30%   27C    P8              27W / 350W |      3MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [9]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    # free unreferenced tensors from the GPU memory.
    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache() 

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
|  1 |  0% | 47% |
|  2 |  0% | 33% |
|  3 |  0% |  0% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
|  1 |  4% | 48% |
|  2 |  0% | 33% |
|  3 |  0% |  0% |


In [10]:
data = pd.read_csv("clean_test_or_not_test.csv" , index_col = 0)
data

,text_clean,label
0,add tests for systemdrawingcommonthis issue tr...,0
1,acctests for packetfabricportresource please k...,0
2,text analytics abstractivesummarizebatchconven...,0
3,see entity recordssee the story in the release...,0
4,editor crash when click mesh instance 3d with ...,0
...,...,...
72563,cdn is downin 987dabdhttpsgithubcomceccunstat...,1
72564,create user registration system using email an...,1
72565,group secure join shows success only after res...,1
72566,plugins not being sourced when using a differe...,1


In [11]:
data_test_set = pd.read_csv("testset_test_or_not_test_clean.csv" , index_col = 0)
data_test_set

,text_clean,label
0,badrecordmac in fips testbuild scan httpsgradl...,0
1,validate shutdown message processing even thou...,0
2,create new flutter project failed on android s...,1
3,upgrade from chromium 103 to chromium 104upgra...,0
4,roachtest followerreadssurvivalzonelocalityreg...,0
...,...,...
8058,autodesk v2 describe the bug autodesk publishe...,1
8059,what happened to sqlite we offer sqlite suppor...,0
8060,setup unittest testing frameworkwe need to uti...,0
8061,use clash for windows cannot be connected in l...,1


In [12]:
# Smaller and faster than distelbert.
base_model_id = "roberta-base"

epochs = 5 #Number of full cyles through the training set.
num_labels = 2 
learning_rate = 5e-5 # Rate the model updates based on the data its trained on.
train_batch_size = 16 # Number of training examples in one iteration.
eval_batch_size = 32 # Number evalutaion examples in on iteratoion.
save_strategy = "no" # Should the model be saved automatically during training.
save_steps = 500 # How often to save the model during training. No effect since no over.
logging_steps = 100
model_dir = "./model" #Where to save model

# Use early stopping to prevent overfitting
#load_best_model_at_end=True
#metric_for_best_model="eval_loss"
#greater_is_better=False

In [13]:
# Split dataframe into three parts: training, validation and testing.
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=42):
    np.random.seed(seed)
    # Shuffle index of dataframe
    perm = np.random.permutation(df.index)
    
    df_length = len(df.index)
    
    # Number of row in training set
    train_end = int(train_percent * df_length)
    # Number of rows in validate set
    validate_end = int(validate_percent * df_length) + train_end
    
    # From start to train end
    train = df.iloc[perm[:train_end]]
    # From train_end to validate_end
    validate = df.iloc[perm[train_end:validate_end]]
    # From validate to the last row in dataframe.
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [14]:
# Drops rows with missing values
data.dropna(inplace=True)

In [15]:
# Resets the index after dropping rows
data.reset_index(inplace=True)
data

,index,text_clean,label
0,0,add tests for systemdrawingcommonthis issue tr...,0
1,1,acctests for packetfabricportresource please k...,0
2,2,text analytics abstractivesummarizebatchconven...,0
3,3,see entity recordssee the story in the release...,0
4,4,editor crash when click mesh instance 3d with ...,0
...,...,...,...
72563,72563,cdn is downin 987dabdhttpsgithubcomceccunstat...,1
72564,72564,create user registration system using email an...,1
72565,72565,group secure join shows success only after res...,1
72566,72566,plugins not being sourced when using a differe...,1


In [16]:
#Drops the index col, better for managint the data.
data.drop(columns= ["index"], inplace = True)

In [17]:
data


,text_clean,label
0,add tests for systemdrawingcommonthis issue tr...,0
1,acctests for packetfabricportresource please k...,0
2,text analytics abstractivesummarizebatchconven...,0
3,see entity recordssee the story in the release...,0
4,editor crash when click mesh instance 3d with ...,0
...,...,...
72563,cdn is downin 987dabdhttpsgithubcomceccunstat...,1
72564,create user registration system using email an...,1
72565,group secure join shows success only after res...,1
72566,plugins not being sourced when using a differe...,1


In [18]:
# 80% trainig, 10% validate, 10% test. Seed 42.
# Test 80-10-10 and 70-15-15
train , validate , test = train_validate_test_split(data)


In [19]:
train.set_index("label" , inplace = True)
validate.set_index("label" , inplace = True)
test.set_index("label" , inplace = True)

In [20]:
test

,text_clean
label,
1,check the bpmn pipelines to see if formsubmiss...
1,add eslint and enforce code style is your feat...
1,rosskopfyxyz is downin 8f48397httpsgithubcomg...
0,time taken by peer link going down when sonic ...
0,test upgrade 42x to 45 time improvement target...
...,...
1,create a release for rke 142my team is waiting...
0,battle error using moves on some togedemarub75...
1,node build fails with latest version bugdescri...


In [21]:
# Convert from Pandas DataFrame to Hugging Face datasets
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(validate)
testds = Dataset.from_pandas(test)

separate_test_set = Dataset.from_pandas(data_test_set)
ds = DatasetDict()

ds["test"] = testds
ds["train"] = tds
ds["validate"] = vds
ds["separate_test_set"] = separate_test_set

ds

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 7258
    })
    train: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 58054
    })
    validate: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 7256
    })
    separate_test_set: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 8063
    })
})

In [22]:
train_dataset = ds["train"]
valid_dataset = ds["validate"]
test_ds = ds["test"]
separate_test_set_dataset = ds["separate_test_set"]

In [23]:
ds["train"][0]

{'text_clean': 'decision tree regressor giving different result for scaled and unscaled datathe decision tree regressor outputs different tree for scaled and unscaled data this is true for the random forest regressor also as per my understanding scaling data should not change the results for decision tree or random forest regressor i am attaching the ipynb file of my analysis in the analysis i have generated a synthetic dataset where the input data is in different scales the decision tree is then made using the unscaled data minmax scaled data and arbitrarily scaled data the arbitrary scaling simply multiplies each input with a suitable value to bring all the inputs in the same order of magnitude it is seen that the tree formed by using the the minmax scaled and arbitrarily scaled data is same but the one formed by the unscaled data is different the same results are reflected when using random forest i am unable to understand this result therefore an explanation would be of help the tr

In [24]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(base_model_id, num_labels=num_labels)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Tokanization

In [27]:
model_ckpt = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [28]:
#Tokenize the dataset to the correct input for the transformer model.
def tokenize(batch):
    return tokenizer(batch["text_clean"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
valid_dataset = valid_dataset.map(tokenize, batched=True, batch_size=len(valid_dataset))
test_dataset = test_ds.map(tokenize, batched=True, batch_size=len(test_ds))
separate_test_set_dataset = separate_test_set_dataset.map(tokenize, batched=True, batch_size=len(separate_test_set_dataset))

Map:   0%|          | 0/58054 [00:00<?, ? examples/s]

Map:   0%|          | 0/7256 [00:00<?, ? examples/s]

Map:   0%|          | 0/7258 [00:00<?, ? examples/s]

Map:   0%|          | 0/8063 [00:00<?, ? examples/s]

In [29]:
training_args = TrainingArguments(
    output_dir=model_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    save_strategy=save_strategy,
    save_steps=save_steps,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    logging_steps=logging_steps,
)

In [30]:
 trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

In [31]:
trainer.train() 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.465100,0.443286,0.835171,[0.82312925 0.84567742],[0.88687062 0.79577465],[0.76793598 0.90225771]
2,0.450500,0.444471,0.831174,[0.81323372 0.84597007],[0.90868825 0.77852349],[0.73592715 0.92621145]
3,0.398700,0.439246,0.843026,[0.83350387 0.85151871],[0.88622941 0.80861599],[0.78669978 0.89922907]
4,0.329500,0.369702,0.870865,[0.86969823 0.87201202],[0.87664704 0.86527514],[0.86285872 0.87885463]
5,0.322400,0.356911,0.877067,[0.87422448 0.87978437],[0.89388697 0.86166843],[0.85540839 0.89867841]


TrainOutput(global_step=18145, training_loss=0.4128472686372338, metrics={'train_runtime': 7139.8157, 'train_samples_per_second': 40.655, 'train_steps_per_second': 2.541, 'total_flos': 7.63732460393472e+16, 'train_loss': 0.4128472686372338, 'epoch': 5.0})

In [32]:
eval_result = trainer.evaluate(eval_dataset=valid_dataset)

In [33]:
for key, value in sorted(eval_result.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.8770672546857773

eval_f1 = [0.87422448 0.87978437]

eval_loss = 0.3569111227989197

eval_precision = [0.89388697 0.86166843]

eval_recall = [0.85540839 0.89867841]

eval_runtime = 54.7927

eval_samples_per_second = 132.426

eval_steps_per_second = 4.143



## Training loss decreases, valdiation loss increases = Overfitting

In [34]:
# Evaluate test data set
test_results = trainer.evaluate(eval_dataset=test_dataset)

In [35]:
for key, value in sorted(test_results.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.8764122347754202

eval_f1 = [0.877241  0.8755722]

eval_loss = 0.35561347007751465

eval_precision = [0.9002809  0.85343429]

eval_recall = [0.85535095 0.89888921]

eval_runtime = 54.6632

eval_samples_per_second = 132.777

eval_steps_per_second = 4.153



# Test set

In [36]:
separate_test_set_results = trainer.evaluate(eval_dataset=separate_test_set_dataset)

In [37]:
for key, value in sorted(separate_test_set_results.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.878333126627806

eval_f1 = [0.87511139 0.88139282]

eval_loss = 0.35675114393234253

eval_precision = [0.90423573 0.85523229]

eval_recall = [0.84780464 0.90920429]

eval_runtime = 60.7217

eval_samples_per_second = 132.786

eval_steps_per_second = 4.15



In [38]:
trainer.save_model(model_dir + "_local") 

In [39]:
from transformers import pipeline
    
classifier = pipeline("text-classification", model="./model_local")

In [40]:
classifier.model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [41]:
classifier("this contain bugs regarding testing")

[{'label': 'LABEL_0', 'score': 0.8654991388320923}]

In [42]:
classifier("this contain bugs regarding automtion not testing")

[{'label': 'LABEL_0', 'score': 0.8445073366165161}]

In [43]:
classifier("this bug has super high impact on the project")

[{'label': 'LABEL_1', 'score': 0.8593593835830688}]

### Delete the dataset with large memory

In [44]:
del valid_dataset

In [45]:
del model

In [46]:
# Free cache
torch.cuda.empty_cache()

In [47]:
!nvidia-smi

Tue Feb 13 22:16:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        On  | 00000000:41:00.0 Off |                  N/A |
| 30%   28C    P8              27W / 350W |      3MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--